문서 유사도 응용
=====
네이버 영화 시놉시스 유사도를 통한 간단한 영화 추천시스템 만들어보기
-----
BoW, TF-IDF에 대해 다시 한번 짚고 넘어가면,
- BoW 방식은 말 그대로 단어가 담긴 가방입니다. 단어 출현 순서가 중요하지 않고 단지 어떤 단어가 쓰였는지가 더 중요합니다.
- TF-IDF는 Term Frequency에 Document Frequency의 역수를 곱한 것입니다. 단어의 빈도만 가지고 계산하게 되면 중요하지 않지만 자주 등장하는 단어가 더 큰 중요도를 가지게 됩니다. 이 영향력을 줄여주기 위해서 IDF를 곱해줍니다.

TF-IDF를 벡터화하고 유클리디안 거리 유사도와 코사인 유사도를 각각 구해보았습니다.<br>

지난 시간에 이어 이번 시간에도 문서 유사도를 실전 응용을 해볼 것입니다.<br>
오늘은 직접 크롤링한 '네이버 영화'의 영화 시놉시스 데이터를 가지고 입력한 영화제목과 유사한 내용의 영화를 소개해주는 모델을 만들어보도록 하겠습니다.<br>
해당 데이터는 '네이버 영화'의 데이터 중 인기순 영화를 1만개 가량 크롤링한 데이터 입니다.

길이가 적당한 코퍼스를 찾으려고 노력하였지만...줄거리가 잘 정리되어 있는 소스를 찾지 못해 영화의 시놉시스 데이터를 활용합니다. <br>
시놉시스 특성상, 굉장히 추상적이고 서론 부분을 요약한 내용이 대부분이기 때문에 눈에 띄는 성능을 보여드리지 못하는 점 죄송합니다! <br>

프로세스는 다음과 같습니다.
- 데이터 불러오기
- 데이터 전처리(중복값 제거 등)
- 텍스트 전처리(명사 중심)
- 코사인 유사도 구하기
- 문서 유사도 기반 추천

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# 디렉토리 확인
%ls

 C 드라이브의 볼륨: SSD
 볼륨 일련 번호: 786C-ABF7

 C:\Users\HONG\Desktop\HONGYP\BOAZ\New 텍스트마이닝\4주차 디렉터리

2020-08-26  오후 12:52    <DIR>          .
2020-08-26  오후 12:52    <DIR>          ..
2020-08-14  오후 02:27    <DIR>          .ipynb_checkpoints
2020-08-14  오후 03:22         4,002,288 4주차 배포파일.zip
2020-08-14  오후 03:13         4,591,768 4주차 실습파일.zip
2020-08-14  오후 02:55            82,545 6강_텍스트 표현 배포용.ipynb
2020-08-14  오후 02:55            11,613 6강_텍스트 표현 실습용.ipynb
2020-08-14  오후 03:12            17,501 7강_문서 유사도 응용_배포용.ipynb
2020-08-14  오후 03:12            17,600 7강_문서 유사도 응용_실습용.ipynb
2020-05-13  오전 12:03            82,664 8강_감성 분석_배포용.ipynb
2020-08-14  오후 02:47            60,175 bigram.png
2020-08-14  오후 02:47            41,954 bow.png
2020-08-11  오후 02:28             3,851 cosine.PNG
2020-08-11  오후 10:55            67,062 crawling.txt
2020-08-14  오후 10:30           639,386 KakaoTalk_20200814_223014226.jpg
2020-08-14  오후 02:46            40,052 trigram.png
2020-08-26  오후 12:52            12,155

In [3]:
# 데이터프레임 불러오기
df = pd.read_csv('영화 시놉시스 데이터.csv', index_col = 0)
df.head()

,영화제목,시놉시스
0,너의 이름은.,"아직 만난 적 없는 너를, 찾고 있어 천년 만에 다가오는 혜성 기적이 시작된다 ..."
1,증인,“목격자가 있어. 자폐아야”신념은 잠시 접어두고 현실을 위해 속물이 되기로 마음먹은...
2,청년경찰,의욕충만 경찰대생 기준(박서준) X 이론백단 경찰대생 희열(강하늘)둘도 없는 친구인...
3,돈,오직 부자가 되고 싶은 꿈을 품고 여의도 증권가에 입성한 신입 주식 브로커 조일현(...
4,비긴 어게인,싱어송라이터인 ‘그레타’(키이라 나이틀리)는 남자친구 ‘데이브’(애덤 리바인)가 메...


In [5]:
# 추가 전처리
# []안을 채워주세요

import re
def extra_process(x):
    x = re.sub("[^\w\s]", " ", x) # 특수문자 제거
    x = re.sub("[접기]", " ", x) # "접기" 라는 단어 제거
    return x

df["시놉시스"] = df["시놉시스"].apply(extra_process)

In [6]:
# DataFrame 확인

df.head()

,영화제목,시놉시스
0,너의 이름은.,아직 만난 적 없는 너를 찾고 있어 천년 만에 다가오는 혜성 적이 시작된다 ...
1,증인,목격자가 있어 자폐아야 신념은 잠시 어두고 현실을 위해 속물이 되 로 마음먹은...
2,청년경찰,의욕충만 경찰대생 준 박서준 X 이론백단 경찰대생 희열 강하늘 둘도 없는 친구인...
3,돈,오직 부자가 되고 싶은 꿈을 품고 여의도 증권가에 입성한 신입 주식 브로커 조일현 ...
4,비긴 어게인,싱어송라이터인 그레타 키이라 나이틀리 는 남자친구 데이브 애덤 리바인 가 메...


In [7]:
# 데이터 정보 확인하기
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10533 entries, 0 to 10532
Data columns (total 2 columns):
영화제목    10533 non-null object
시놉시스    10533 non-null object
dtypes: object(2)
memory usage: 246.9+ KB


In [8]:
df.isnull().sum()

영화제목    0
시놉시스    0
dtype: int64

In [9]:
# 데이터 예시
df.head(10)

,영화제목,시놉시스
0,너의 이름은.,아직 만난 적 없는 너를 찾고 있어 천년 만에 다가오는 혜성 적이 시작된다 ...
1,증인,목격자가 있어 자폐아야 신념은 잠시 어두고 현실을 위해 속물이 되 로 마음먹은...
2,청년경찰,의욕충만 경찰대생 준 박서준 X 이론백단 경찰대생 희열 강하늘 둘도 없는 친구인...
3,돈,오직 부자가 되고 싶은 꿈을 품고 여의도 증권가에 입성한 신입 주식 브로커 조일현 ...
4,비긴 어게인,싱어송라이터인 그레타 키이라 나이틀리 는 남자친구 데이브 애덤 리바인 가 메...
5,미스 슬로운,승률 100 를 자랑하는 최고의 로비스트 슬로운 제시카 차스테인 총 규제 ...
6,신과함께-죄와 벌,저승 법에 의하면 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다 살인...
7,아이 필 프리티,뛰어난 패션센스에 매력적인 성격이지만 통통한 몸매가 불만인 르네 하아 예뻐지...
8,리틀 포레스트,시험 연애 취업 뭐하나 뜻대로 되지 않는 일상을 잠시 멈추고고향으로 돌아온 혜...
9,어바웃 타임,모태솔로 팀 돔놀 글리슨 은 성인이 된 날 아버지 빌 나이 로부터 놀랄만한 가문의...


In [10]:
# 영화제목 기준 중복값 확인, 중복값 중 디폴트 keep=first이므로 처음거 킵하고 나머지 버림
df[df.duplicated(subset = '영화제목')]

,영화제목,시놉시스
58,해리 포터와 마법사의 돌(1편),해리 포터 다니엘 래드클리프 분 는 위압적인 버논 숙부 리챠드 그리피스 분 와 냉담...
63,어바웃 타임,모태솔로 팀 돔놀 글리슨 은 성인이 된 날 아버지 빌 나이 로부터 놀랄만한 가문의...
106,어벤져스,지구의 안보가 위협당하는 위 의 상황에서 슈퍼히어로들을 불러모아 세상을 구하는 일...
126,"해리 포터와 죽음의 성물 - 2부(7-2편, 자막판)",모든 것을 끝낼 최후의 전투 판타지의 아름다운 역사가 드디어 마침표를 찍는다 덤블...
128,해리 포터와 비밀의 방(2편),해리 포터에겐 이번 여름방학이 별로 즐겁질 못했다 마법이라면 질색을 하는 페투니아...
129,플립,새로 이사 온 미소년 브라이스를 보고 첫눈에 사랑을 직감한 7살 소녀 줄리 솔직하...
148,해리 포터와 아즈카반의 죄수(3편),13세가 된 해리 포터 다니엘 래드클래프 는 아버지의 험담을 하는 이모부의 누이...
157,노트북,17살 노아 는 밝고 순수한 앨리 를 보고 첫눈에 반한다 빠른 속도로 서로...
158,주토피아,누구나 살고 싶은 도시 1위 주토피아 연쇄 실종 사건 발생 미치도록 잡고 싶었...
173,인셉션,타인의 꿈에 들어가 생각을 훔치는 특수 보안요원 코브 그를 이용해 라이벌 업의 ...


In [11]:
# 영화제목 기준 중복값 제거
df.drop_duplicates(subset = '영화제목', keep = 'first', inplace = True)

In [12]:
# 인덱스 재설정
df.reset_index(drop = True, inplace = True)
df

,영화제목,시놉시스
0,너의 이름은.,아직 만난 적 없는 너를 찾고 있어 천년 만에 다가오는 혜성 적이 시작된다 ...
1,증인,목격자가 있어 자폐아야 신념은 잠시 어두고 현실을 위해 속물이 되 로 마음먹은...
2,청년경찰,의욕충만 경찰대생 준 박서준 X 이론백단 경찰대생 희열 강하늘 둘도 없는 친구인...
3,돈,오직 부자가 되고 싶은 꿈을 품고 여의도 증권가에 입성한 신입 주식 브로커 조일현 ...
4,비긴 어게인,싱어송라이터인 그레타 키이라 나이틀리 는 남자친구 데이브 애덤 리바인 가 메...
5,미스 슬로운,승률 100 를 자랑하는 최고의 로비스트 슬로운 제시카 차스테인 총 규제 ...
6,신과함께-죄와 벌,저승 법에 의하면 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다 살인...
7,아이 필 프리티,뛰어난 패션센스에 매력적인 성격이지만 통통한 몸매가 불만인 르네 하아 예뻐지...
8,리틀 포레스트,시험 연애 취업 뭐하나 뜻대로 되지 않는 일상을 잠시 멈추고고향으로 돌아온 혜...
9,어바웃 타임,모태솔로 팀 돔놀 글리슨 은 성인이 된 날 아버지 빌 나이 로부터 놀랄만한 가문의...


In [13]:
# TF-IDF 벡터화 
vectorizer=TfidfVectorizer(min_df=1, ngram_range=(1,1))
features=vectorizer.fit_transform(df['시놉시스'])

In [14]:
# feature 이름 불러 오기
feature_names=vectorizer.get_feature_names()

In [15]:
# DTM 생성
dtm_np=np.array(features.todense())

In [16]:
# DataFrame 확인
pd.DataFrame(data = dtm_np, columns = feature_names)

,00,000,000km,000km의,000개의,000년,000년간,000년경,000년의,000달러,...,힙합씬에서,힙합을,힙합의,힙합키드들은,힙합프로댄스팀,힝클리,雷峰怪蹟,女형사,流來覺,流砂
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# 코사인 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim=cosine_similarity(dtm_np, dtm_np)

In [18]:
cosine_sim

array([[1.        , 0.00949305, 0.0189276 , ..., 0.        , 0.01105723,
        0.        ],
       [0.00949305, 1.        , 0.00417506, ..., 0.007475  , 0.00462524,
        0.        ],
       [0.0189276 , 0.00417506, 1.        , ..., 0.00113903, 0.00160581,
        0.        ],
       ...,
       [0.        , 0.007475  , 0.00113903, ..., 1.        , 0.        ,
        0.        ],
       [0.01105723, 0.00462524, 0.00160581, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [10]:
# 영화제목 기준 인덱스(복수형 indices) 설정 인덱스
indices = pd.Series(df.index, index=df['영화제목'])
indices

In [11]:
# 영화제목 확인 하기
set(df['영화제목'])

In [12]:
# 마음에 드는 영화 인덱스 추출
idx=indices['트와일라잇']

In [13]:
# 선택한 영화와 다른 영화의 유사도 확인
sim_scores=list(enumerate(cosine_sim[idx])); sim_scores

In [14]:
# 유사도가 높은 순으로 영화 정렬
sim_scores=sorted(sim_scores, key=lambda x: x[1], reverse=True); sim_scores

In [16]:
# 유사도 높은 10개의 영화
sim_scores=sim_scores[1:11]

In [17]:
# 가장 유사한 10개의 영화의 인덱스 
book_indices=[i[0] for i in sim_scores]

In [18]:
# 유사한 영화 제목 출력
df['영화제목'].iloc[book_indices]

In [29]:
# 추천 함수 만들기 (제목을 입력하면 위의 작업을 자동화합니다.)

def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 인덱스를 가지고 옵니다.
    idx = indices[title]

    # 검색한 영화와 모든 영화의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도가 높은 순으로 영화를 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 가지고 옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    book_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 반환합니다.
    return df['영화제목'].iloc[book_indices]

# 코드 출처: https://wikidocs.net/24603 '딥 러닝을 이용한 자연어 처리 입문'

In [ ]:
# 영화 검색
title = '어벤져스' 
get_recommendations(title)

텍스트 전처리
-----
아마 위와 같이 작업을 했다면 원하는 만큼 결과가 나오지 않았을 것입니다. 전처리를 제대로 해주지 않았기 때문인데요,<br>
기존 실습내용인 책을 활용하였을 때, 전처리 후 성능 향상이 가시적이지만 이번 실습에서 사용하는 영화 시놉시스 데이터의 경우 그렇지는 않습니다ㅠ <br>
전처리를 진행하고 다시 유사도 기반 추천 모델을 만들어보겠습니다.<br>
텍스트 전처리 과정은 다음과 같습니다.
- Cleaning: 불필요한 기호들을 제거합니다. (위에서 진행하였습니다.)
- Tokenizing: 문장의 단어들을 쪼갭니다.
- Tagging: 쪼개진 단어에 품사를 태깅합니다.
- Removing Stopwords: 핵심적이지 않은, 불필요한 단어들을 제거합니다.

In [75]:
# Okt 불러오기
import konlpy.tag
okt=konlpy.tag.Okt()

In [76]:
# 명사 단어만 남기기 (okt.nouns 활용) okt.nouns 하면 리스트형태로 반환될것
df['시놉시스']=df['시놉시스'].apply(lambda x : okt.nouns(x))

In [77]:
# 리스트를 문장으로 합치기 (join 활용)
df['시놉시스']=df['시놉시스'].apply(lambda x : " ".join(x))

In [ ]:
df['시놉시스']

In [79]:
# TF-IDF 벡터화
vectorizer=TfidfVectorizer(min_df=1, ngram_range=(1,1))
features=vectorizer.fit_transform(df['줄거리'])

In [80]:
# feature 이름 불러 오기
feature_names=vectorizer.get_feature_names()

In [81]:
# DTM 생성
dtm_np=np.array(features.todense())

In [19]:
# DataFrame 확인
sim_df = pd.DataFrame(data = dtm_np, columns = feature_names)
sim_df

In [20]:
# 불용어 찾아보기 
feature_names[:1000]

In [21]:
# 불용어 제거
stopwords = []
for stopword in stopwords:
    del sim_df[stopword] # 컬럼 삭제

In [85]:
# 코사인 유사도 다시 구하기
cosine_sim = cosine_similarity(sim_df, sim_df)

In [22]:
# 영화 검색
get_recommendations('월드워Z')